In [ ]:
# Copyright (c) 2024 Microsoft Corporation.
# Licensed under the MIT License.

In [1]:
import os

import pandas as pd
import tiktoken

from graphrag.query.context_builder.entity_extraction import EntityVectorStoreKey
from graphrag.query.indexer_adapters import (
    read_indexer_covariates,
    read_indexer_entities,
    read_indexer_relationships,
    read_indexer_reports,
    read_indexer_text_units,
)
from graphrag.query.input.loaders.dfs import (
    store_entity_semantic_embeddings,
)
from graphrag.query.llm.oai.chat_openai import ChatOpenAI
from graphrag.query.llm.oai.embedding import OpenAIEmbedding
from graphrag.query.llm.oai.typing import OpenaiApiType
from graphrag.query.question_gen.local_gen import LocalQuestionGen
from graphrag.query.structured_search.local_search.mixed_context import (
    LocalSearchMixedContext,
)
from graphrag.query.structured_search.local_search.search import LocalSearch
from graphrag.vector_stores.lancedb import LanceDBVectorStore

## Local Search Example

Local search method generates answers by combining relevant data from the AI-extracted knowledge-graph with text chunks of the raw documents. This method is suitable for questions that require an understanding of specific entities mentioned in the documents (e.g. What are the healing properties of chamomile?).

### Load text units and graph data tables as context for local search

- In this test we first load indexing outputs from parquet files to dataframes, then convert these dataframes into collections of data objects aligning with the knowledge model.

### Load tables to dataframes

In [2]:
INPUT_DIR = "./inputs/operation dulce"
LANCEDB_URI = f"{INPUT_DIR}/lancedb"

COMMUNITY_REPORT_TABLE = "create_final_community_reports"
ENTITY_TABLE = "create_final_nodes"
ENTITY_EMBEDDING_TABLE = "create_final_entities"
RELATIONSHIP_TABLE = "create_final_relationships"
COVARIATE_TABLE = "create_final_covariates"
TEXT_UNIT_TABLE = "create_final_text_units"
COMMUNITY_LEVEL = 2

#### Read entities

In [3]:
# read nodes table to get community and degree data
entity_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_TABLE}.parquet")
entity_embedding_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_EMBEDDING_TABLE}.parquet")

entities = read_indexer_entities(entity_df, entity_embedding_df, COMMUNITY_LEVEL)

# load description embeddings to an in-memory lancedb vectorstore
# to connect to a remote db, specify url and port values.
description_embedding_store = LanceDBVectorStore(
    collection_name="entity_description_embeddings",
)
description_embedding_store.connect(db_uri=LANCEDB_URI)
entity_description_embeddings = store_entity_semantic_embeddings(
    entities=entities, vectorstore=description_embedding_store
)

print(f"Entity count: {len(entity_df)}")
entity_df.head()

Entity count: 434


[2024-07-09T02:22:59Z WARN  lance::dataset] No existing dataset at /media/gpt4-pdf-chatbot-langchain/graphrag/examples_notebooks/inputs/operation dulce/lancedb/entity_description_embeddings.lance, it will be created


,level,title,type,description,source_id,community,degree,human_readable_id,id,size,graph_embedding,entity_type,top_level_node_id,x,y
0,0,ALEX MERCER,PERSON,Alex Mercer is a character with a military bac...,"00fafabae48948779fee2afe600f5143,1e433d6b30887...",1,9,0,b45241d70f0e43fca764df95b2b81f77,9,None,None,b45241d70f0e43fca764df95b2b81f77,0,0
1,0,TAYLOR CRUZ,PERSON,Taylor Cruz is a character who plays a pivotal...,"00fafabae48948779fee2afe600f5143,1e433d6b30887...",1,12,1,4119fd06010c494caa07f439b333f4c5,12,None,None,4119fd06010c494caa07f439b333f4c5,0,0
2,0,JORDAN HAYES,PERSON,Dr. Jordan Hayes is a key character in a narra...,"00fafabae48948779fee2afe600f5143,2cf7a230c367a...",1,9,2,d3835bf3dda84ead99deadbeac5d0d7d,9,None,None,d3835bf3dda84ead99deadbeac5d0d7d,0,0
3,0,SAM RIVERA,PERSON,Sam Rivera is a character renowned for their t...,"00fafabae48948779fee2afe600f5143,1e433d6b30887...",1,15,3,077d2820ae1845bcbb1803379a3d1eae,15,None,None,077d2820ae1845bcbb1803379a3d1eae,0,0
4,0,PARANORMAL MILITARY SQUAD,ORGANIZATION,The Paranormal Military Squad is an elite clan...,"00fafabae48948779fee2afe600f5143,1e433d6b30887...",1,14,4,3671ea0dd4e84c1a9b02c5ab2c8f4bac,14,None,None,3671ea0dd4e84c1a9b02c5ab2c8f4bac,0,0


#### Read relationships

In [4]:
relationship_df = pd.read_parquet(f"{INPUT_DIR}/{RELATIONSHIP_TABLE}.parquet")
relationships = read_indexer_relationships(relationship_df)

print(f"Relationship count: {len(relationship_df)}")
relationship_df.head()

Relationship count: 276


,source,target,weight,description,text_unit_ids,id,human_readable_id,source_degree,target_degree,rank
0,ALEX MERCER,TAYLOR CRUZ,7.0,Alex Mercer and Taylor Cruz are integral membe...,"[00fafabae48948779fee2afe600f5143, 1e433d6b308...",b35c3d1a7daa4924b6bdb58bc69c354d,0,9,12,21
1,ALEX MERCER,TAYLOR CRUZ,7.0,Alex Mercer and Taylor Cruz are integral membe...,"[00fafabae48948779fee2afe600f5143, 1e433d6b308...",b35c3d1a7daa4924b6bdb58bc69c354d,0,9,12,21
2,ALEX MERCER,TAYLOR CRUZ,7.0,Alex Mercer and Taylor Cruz are integral membe...,"[00fafabae48948779fee2afe600f5143, 1e433d6b308...",b35c3d1a7daa4924b6bdb58bc69c354d,0,9,12,21
3,ALEX MERCER,TAYLOR CRUZ,7.0,Alex Mercer and Taylor Cruz are integral membe...,"[00fafabae48948779fee2afe600f5143, 1e433d6b308...",b35c3d1a7daa4924b6bdb58bc69c354d,0,9,12,21
4,ALEX MERCER,JORDAN HAYES,6.0,Alex Mercer and Jordan Hayes are colleagues wh...,"[00fafabae48948779fee2afe600f5143, 2cf7a230c36...",a97e2ecd870944cfbe71c79bc0fcc752,1,9,9,18


In [5]:
covariate_df = pd.read_parquet(f"{INPUT_DIR}/{COVARIATE_TABLE}.parquet")

claims = read_indexer_covariates(covariate_df)

print(f"Claim records: {len(claims)}")
covariates = {"claims": claims}

Claim records: 89


#### Read community reports

In [6]:
report_df = pd.read_parquet(f"{INPUT_DIR}/{COMMUNITY_REPORT_TABLE}.parquet")
reports = read_indexer_reports(report_df, entity_df, COMMUNITY_LEVEL)

print(f"Report records: {len(report_df)}")
report_df.head()

Report records: 6


,community,full_content,level,rank,title,rank_explanation,summary,findings,full_content_json,id
0,4,# Dulce Base and the Paranormal Military Squad...,1,8.5,Dulce Base and the Paranormal Military Squad: ...,The impact severity rating is high due to the ...,"The community is centered around Dulce Base, a...",[{'explanation': 'Dulce Base is the primary lo...,"{\n ""title"": ""Dulce Base and the Paranormal...",6f8ba6b6-506e-46c1-83ce-982d59622554
1,5,# Sam Rivera and the Paranormal Military Squad...,1,7.5,Sam Rivera and the Paranormal Military Squad a...,The impact severity rating is high due to the ...,"The community is centered around Sam Rivera, a...",[{'explanation': 'Sam Rivera is recognized for...,"{\n ""title"": ""Sam Rivera and the Paranormal...",418f4536-d673-4212-8a7c-ca1aac547d0f
2,0,# Dulce Base and the Paranormal Military Squad...,0,8.5,Dulce Base and the Paranormal Military Squad Team,The impact severity rating is high due to the ...,Dulce Base serves as the operational hub for t...,[{'explanation': 'The Paranormal Military Squa...,"{\n ""title"": ""Dulce Base and the Paranormal...",251df57e-fd49-49a7-b262-ccaff95d7a51
3,1,# Dulce Base and the Paranormal Military Squad...,0,8.5,Dulce Base and the Paranormal Military Squad: ...,The impact severity rating is high due to the ...,"The community is centered around Dulce Base, a...",[{'explanation': 'Sam Rivera is recognized for...,"{\n ""title"": ""Dulce Base and the Paranormal...",6e536385-8056-4a82-8670-c0ccaf007fb4
4,2,# Dulce Base: Extraterrestrial Research and Co...,0,8.5,Dulce Base: Extraterrestrial Research and Comm...,The impact severity rating is high due to the ...,Dulce Base is a highly classified facility in ...,[{'explanation': 'Dulce Base is the epicenter ...,"{\n ""title"": ""Dulce Base: Extraterrestrial ...",660c8fba-b65f-4fcd-9c98-203600cf1981


#### Read text units

In [7]:
text_unit_df = pd.read_parquet(f"{INPUT_DIR}/{TEXT_UNIT_TABLE}.parquet")
text_units = read_indexer_text_units(text_unit_df)

print(f"Text unit records: {len(text_unit_df)}")
text_unit_df.head()

Text unit records: 12


,id,text,n_tokens,document_ids,entity_ids,relationship_ids,covariate_ids
0,2cf7a230c367a2dfaf0fc3c903eb8948,# Operation: Dulce\n\n## Chapter 1\n\nThe thru...,2500,[958fdd043f17ade63cb13570b59df295],"[b45241d70f0e43fca764df95b2b81f77, 4119fd06010...","[b35c3d1a7daa4924b6bdb58bc69c354d, a97e2ecd870...","[ad5a2020-cdec-4982-acdf-dbe5ee530066, 9d8a0fe..."
1,6d1255303acb7c9dc951cb0f5fc3042c,be the same.\n\n\*\n\nThe sense of foreboding...,2500,[958fdd043f17ade63cb13570b59df295],"[b45241d70f0e43fca764df95b2b81f77, 4119fd06010...","[b35c3d1a7daa4924b6bdb58bc69c354d, a97e2ecd870...","[5d1c9126-c48d-4755-9f9c-f739c823f95f, ec64a42..."
2,e841f178310356740b2ee9101d12c97f,". ""Your take on these signal inconsistencies?""...",2500,[958fdd043f17ade63cb13570b59df295],"[b45241d70f0e43fca764df95b2b81f77, 4119fd06010...","[b35c3d1a7daa4924b6bdb58bc69c354d, a97e2ecd870...","[0b22a34b-32e9-46a4-a0e8-d3d5466eba15, 7e14972..."
3,f36d96862b9366d7240b5c7ceb04f12b,", absorbed in the bewilderment of contact, whi...",2500,[958fdd043f17ade63cb13570b59df295],"[b45241d70f0e43fca764df95b2b81f77, 4119fd06010...","[b35c3d1a7daa4924b6bdb58bc69c354d, 09f18f81442...","[9cd6d645-ab97-4b39-b02e-647cea9b5545, 50dc124..."
4,f7d43808d2fb452cd953bf50c6de6bd4,"were at once coherent and enigmatic: ""*Voyage...",2500,[958fdd043f17ade63cb13570b59df295],"[b45241d70f0e43fca764df95b2b81f77, 4119fd06010...","[e02be3e37ca0454883a4c1fd859c24bb, 1dbc51475cb...",[87cf5900-6211-4e04-9115-50f3617c88b4]


In [8]:
api_key =  os.environ["ZHIPUAI_API_KEY"]

llm = ChatOpenAI(
    api_key=api_key,
    model="glm-4-airx",
    api_base="https://open.bigmodel.cn/api/paas/v4",
    api_type=OpenaiApiType.OpenAI,  # OpenaiApiType.OpenAI or OpenaiApiType.AzureOpenAI
    max_retries=20,
)

token_encoder = tiktoken.get_encoding("cl100k_base")

text_embedder = OpenAIEmbedding(
    api_key=api_key,
    api_base="https://open.bigmodel.cn/api/paas/v4",
    api_type=OpenaiApiType.OpenAI,
    model="embedding-2",
    deployment_name="embedding-2",
    max_retries=20,
)

### Create local search context builder

In [9]:
context_builder = LocalSearchMixedContext(
    community_reports=reports,
    text_units=text_units,
    entities=entities,
    relationships=relationships,
    covariates=covariates,
    entity_text_embeddings=description_embedding_store,
    embedding_vectorstore_key=EntityVectorStoreKey.ID,  # if the vectorstore uses entity title as ids, set this to EntityVectorStoreKey.TITLE
    text_embedder=text_embedder,
    token_encoder=token_encoder,
)

### Create local search engine

In [10]:
# text_unit_prop: proportion of context window dedicated to related text units
# community_prop: proportion of context window dedicated to community reports.
# The remaining proportion is dedicated to entities and relationships. Sum of text_unit_prop and community_prop should be <= 1
# conversation_history_max_turns: maximum number of turns to include in the conversation history.
# conversation_history_user_turns_only: if True, only include user queries in the conversation history.
# top_k_mapped_entities: number of related entities to retrieve from the entity description embedding store.
# top_k_relationships: control the number of out-of-network relationships to pull into the context window.
# include_entity_rank: if True, include the entity rank in the entity table in the context window. Default entity rank = node degree.
# include_relationship_weight: if True, include the relationship weight in the context window.
# include_community_rank: if True, include the community rank in the context window.
# return_candidate_context: if True, return a set of dataframes containing all candidate entity/relationship/covariate records that
# could be relevant. Note that not all of these records will be included in the context window. The "in_context" column in these
# dataframes indicates whether the record is included in the context window.
# max_tokens: maximum number of tokens to use for the context window.


local_context_params = {
    "text_unit_prop": 0.5,
    "community_prop": 0.1,
    "conversation_history_max_turns": 5,
    "conversation_history_user_turns_only": True,
    "top_k_mapped_entities": 10,
    "top_k_relationships": 10,
    "include_entity_rank": True,
    "include_relationship_weight": True,
    "include_community_rank": False,
    "return_candidate_context": False,
    "embedding_vectorstore_key": EntityVectorStoreKey.ID,  # set this to EntityVectorStoreKey.TITLE if the vectorstore uses entity title as ids
    "max_tokens": 12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
}

llm_params = {
    "max_tokens": 2_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 1000=1500)
    "temperature": 0.1,
}

In [11]:
search_engine = LocalSearch(
    llm=llm,
    context_builder=context_builder,
    token_encoder=token_encoder,
    llm_params=llm_params,
    context_builder_params=local_context_params,
    response_type="multiple paragraphs",  # free form text describing the response type and format, can be anything, e.g. prioritized list, single paragraph, multiple paragraphs, multiple-page report
)

### Run local search on sample queries

In [12]:
result = await search_engine.asearch("Tell me about Agent Mercer")
print(result.response)

ValueError: Query vector size 1024 does not match index column size 1536

In [ ]:
question = "Tell me about Dr. Jordan Hayes"
result = await search_engine.asearch(question)
print(result.response)

#### Inspecting the context data used to generate the response

In [ ]:
result.context_data["entities"].head()

In [ ]:
result.context_data["relationships"].head()

In [ ]:
result.context_data["reports"].head()

In [ ]:
result.context_data["sources"].head()

In [ ]:
if "claims" in result.context_data:
    print(result.context_data["claims"].head())

### Question Generation

This function takes a list of user queries and generates the next candidate questions.

In [ ]:
question_generator = LocalQuestionGen(
    llm=llm,
    context_builder=context_builder,
    token_encoder=token_encoder,
    llm_params=llm_params,
    context_builder_params=local_context_params,
)

In [ ]:
question_history = [
    "Tell me about Agent Mercer",
    "What happens in Dulce military base?",
]
candidate_questions = await question_generator.agenerate(
    question_history=question_history, context_data=None, question_count=5
)
print(candidate_questions.response)